# Feedback
## Exercise 1
1. no sé que hicieron para cargar los datos, pero es tremendamente lento. Observaciones:
    - los preprocessors usan datos de la validación!!
    - en general no está limpio. Aplican one hot al final otra vez ... no sé que es esto. Preciso para esto usamos pipelines ... 0.5/1.5
2. ok. 0.5/0.5
    
## Exercise 2
1. good.1.5/1.5

2. good. 0.5/0.5


## Exercise 3
1. bien, pero se debería retornar una columna *por cada atributo* y no hacer un resumen de los atributos en una columna. 0.75/1
2. esto es confuso. Realmente no entiendo esta función. Aparentemente siempre retorna 1 en el score. El wrapping aparantamente usa todos los datos, lo cual obviamente está prohibido. 0.25/1
3. no me queda clara esta interpretación. Al parecer pierden 10% de accuracy en madelon en el test set. Aquí hay un problema. 0.5/1

Total: 0.5 + 0.5 + 1.5 +0.5 + 0.75 + 0.25 + 0.5 = 4.5/7

# Universidad de La Sabana 

**Felix Mohr**

## Facultad de Ingenería 2022.2
Supervised Machine Learning


- Gerson Barrera
- Santiago Merchán
- Rodrigo Carranza

On this sheet, we are working with the datasets mushroom, madelon and amazon-commerce-reviews from openml.org. You can use the API from openml.org to download these datasets (ids 24,1485, and 1457)

Libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.linalg
import scipy.stats
from scipy.stats import norm as normal
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.discriminant_analysis
from sklearn.model_selection import train_test_split
import sklearn.tree
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import sklearn.datasets as dfs
import sklearn.metrics
from numpy import percentile
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from pprint import pprint
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
from numpy import linspace
import random
%matplotlib inline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import power_transform 
from sklearn.preprocessing import Normalizer

In [2]:
#Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


#Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss

#Model Selection
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

import warnings
warnings.filterwarnings("ignore")

import sklearn.tree
import pandas as pd
import sklearn.datasets
import random
from random import seed
from random import randint
import numpy as np
import sklearn.metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector

Data Ingestion

In [4]:
import openml

# List all datasets and their properties
openml.datasets.list_datasets(output_format="dataframe")

# Get dataset by ID
mushroom = openml.datasets.get_dataset(24)
madelon = openml.datasets.get_dataset(1485)
amazon = openml.datasets.get_dataset(1457)


# Get the data itself as a dataframe (or otherwise)
mushroom_df, y, _, _  = mushroom.get_data(dataset_format="dataframe")
madelon_df, y, _, _  = madelon.get_data(dataset_format="dataframe")
amazon_df, y, _, _  = amazon.get_data(dataset_format="dataframe")

KeyboardInterrupt: 

In [ ]:
X_madelon, y_madelon = madelon_df.drop("Class", axis=1), madelon_df["Class"].astype(str)
X_amazon, y_amazon = amazon_df.drop("Class", axis=1), amazon_df["Class"].astype(str)

#**Exercise 1** (Pre-Processing)

1. Write a function get_best_encoding(df, label, learner, scoring), where df is a pandas dataframe and label is the target column. The function should look at combinations of: 

- dropping or keeping a column for the first value in each category 
- every technique of imputation (if the dataset has missing data).

  For each combination, mount a pipeline with the respective pre-processing steps and learner, and check the score of a cross-validation given the scoring function. Return the pair X, y, where X is the encoding for which the best result was obtained. 

2. Apply the function to the mushroom dataset once for kNN and once for a decision tree. Look at the class distribution and choose an appropriate scoring function (justify yourchoice!)

1.1

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


def one_hot(df, drop, handle_unknown):
  
  #one_hot = OneHotEncoder(drop="first")
  #one_hot_df =  pd.DataFrame( one_hot.fit_transform(X[X.columns]))

  encoder = OneHotEncoder(drop=drop, handle_unknown=handle_unknown,  sparse=False).fit(df[df.columns])
  test = pd.DataFrame(encoder.transform(df[df.columns]))
  test.columns = encoder.get_feature_names(df.columns)

  return test




def label_encoder(X, y):
  Encoder_X = LabelEncoder() 
  for col in X.columns:
      X[col] = Encoder_X.fit_transform(X[col])

  Encoder_y=LabelEncoder()
  y = Encoder_y.fit_transform(y)
  return X, y

def get_column_types(df):
  A = df.select_dtypes(include='number').columns
  print("Numeric attributes: ", A)
  B = df.select_dtypes(exclude='number').columns
  #B = df.select_dtypes(['category']).columns
  print("Categorical attributes: ", B)
  return A, B

In [ ]:
def get_best_encoding(df, label, lerner, scoring):

  X_ = df.drop(label, axis=1)
  y_ = df[label]
  result = []
  drops = []
  unknows = []

  a, b = get_column_types(X_)
  matrix_elements = []

  drop_list = [None, "first", "if_binary"]
  handle_unknown = ["error", "ignore"]
  imputers_list = ['most_frequent', 'constant']

  for i in drop_list:
      for j in imputers_list:

        # Transformaciones para las variables numéricas
        numeric_transformer = Pipeline(
                                steps=[
                                    ('imputer', SimpleImputer(strategy='median')),
                                    ('scaler', StandardScaler())
                                ]
                              )


        # Transformaciones para las variables categóricas
        categorical_transformer = Pipeline(
                                    steps=[
                                        ('imputer', SimpleImputer(strategy=j)),
                                        ('onehot', OneHotEncoder(drop=i, handle_unknown='ignore'))
                                    ]
                                  )
        if len(a)>0:
          preprocessor = ColumnTransformer(
                              transformers=[
                                  ('numeric', numeric_transformer, a),
                                  ('cat', categorical_transformer, b)
                              ],
                              remainder='passthrough'
                          )
          
        else:
          preprocessor = ColumnTransformer(
                              transformers=[
                                  ('cat', categorical_transformer, b)
                              ],
                              remainder='passthrough'
                          )      
        
        X_train_prep = preprocessor.fit_transform(X_)
        #X_test_prep  = preprocessor.transform(X_test)

        encoded_cat = preprocessor.named_transformers_['cat']['onehot']\
                      .get_feature_names(b)
        labels = np.concatenate([a, encoded_cat])
        datos_train_prep = preprocessor.transform(X_)
        
        datos_train_prep = pd.DataFrame(datos_train_prep, columns=labels)
        clf = lerner
        res = np.mean(sklearn.model_selection.cross_validate(estimator=clf, X=datos_train_prep, y=y_, cv=5, n_jobs=4, scoring = scoring)['test_score'])


        result.append(res)
        drops.append(i)
        unknows.append(j)


        matrix_elements.append((res, i, j))


  df = pd.DataFrame(data = matrix_elements, columns = ['score', 'imputation', 'drop'])
  df.score = round(df.score, 2)
  df_matriz = pd.pivot_table(df, values='score', index=['drop'],  columns=['imputation'])

  a, b = df_matriz.stack().idxmax()
  s = df_matriz.loc[[a], [b]].values
  print("La mejor configuración posible con este Dataset es con drop:", a,"\n",  "Handle unknown:", b,"\n", "Score:", s ) 
  X_  = one_hot(X_, a, b)
  return X_, y_

In [ ]:
def get_best_encoding(df, label, lerner, scoring):

  drop_list = [None, "first", "if_binary"]
  handle_unknown = ["error", "ignore"]

  a, b = get_column_types(df)

  for i in a:
    df[i] = df[i].fillna(df[i].mean()) # Imputation for Numerical Data

  for i in b:
    df[i] = df[i].fillna(df[i].mode()) # Imputation for Categorical Data


  X_ = df.drop(label, axis=1)
  y_ = df[label]

  result = []
  drops = []
  unknows = []

  matrix_elements = []
  for i in drop_list:
    for j in handle_unknown:

      X_  = one_hot(X_, i, j)
      X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.3)
      
      clf = lerner.fit(X_train,y_train)
      predicted = clf.predict(X_test)
      predicted_proba = clf.predict(X_test)
      res = np.mean(sklearn.model_selection.cross_validate(estimator=clf, X=X_train, y=y_train, cv=5, n_jobs=4, scoring = scoring)['test_score'])

      result.append(res)
      drops.append(i)
      unknows.append(j)

      matrix_elements.append((res, i, j))
      
      
  df = pd.DataFrame(data = matrix_elements, columns = ['score', 'drop_criterio', 'unknown'])
  df.score = round(df.score, 2)
  df_matriz = pd.pivot_table(df, values='score', index=['drop_criterio'],  columns=['unknown'])

  a, b = df_matriz.stack().idxmax()
  s = df_matriz.loc[[a], [b]].values
  print("La mejor configuración posible con este Dataset es con drop:", a,"\n",  "Handle unknown:", b,"\n", "Score:", s ) 
  X_  = one_hot(X_, a, b)
  return X_, y_

In [ ]:
X_mushroom, y_mushroom = get_best_encoding(mushroom_df, "class",KNeighborsClassifier(), "accuracy" )

Numeric attributes:  Index([], dtype='object')
Categorical attributes:  Index(['cap-shape', 'cap-surface', 'cap-color', 'bruises%3F', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat', 'class'],
      dtype='object')
La mejor configuración posible con este Dataset es con drop: first 
 Handle unknown: error 
 Score: [[1.]]


In [ ]:
X_mushroom, y_mushroom = get_best_encoding(mushroom_df, "class",DecisionTreeClassifier(), "accuracy" )

Numeric attributes:  Index([], dtype='object')
Categorical attributes:  Index(['cap-shape', 'cap-surface', 'cap-color', 'bruises%3F', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat', 'class'],
      dtype='object')
La mejor configuración posible con este Dataset es con drop: first 
 Handle unknown: error 
 Score: [[1.]]



Se utilizó como métrica de Scoring el "Accuracy" teniendo en cuenta que es la medida más directa de la calidad de los clasificadores, en este caso Knn y Árboles de Decisión. Además, al ser una base de datos balanceada, es una buena idea usar esta métrica. El accuracy de este valor entre 0 y 1 se toma el mayor para seleccionar la mejor combinación de Parámetros o Pipeline. En este caso al ser tan sencilla la clasificación, la métrica es muy cercana a 1 con ambos clasificadores. 



# **Exercise 2** (Filtering)

1. Write a function analyze_filtering_benefits(X, y) in which you plug a feature se-lector (sklearn.feature_selection.SelectPercentile) together with (i) kNN and (i) DT into two pipelines. 
Compare the cross-validation performances of kNN/DT when filtering is used or notused on the given data, and report your findings. 
2. Run the filtering analysis on the madelon and amazon-commerce-reviewsdatasets. In-terpret your observations!

2.1

In [ ]:
from sklearn import model_selection
from sklearn import pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile, f_classif

#X = madelon_df.drop("Class", axis=1)
#y = madelon_df["Class"]
def analyze_filtering_benefits(X, y):
    #scoring=accuracy_score
    fea_selector=SelectPercentile(f_classif)
    modelo_filtrado_dt=Pipeline([("filter",fea_selector),("model",DecisionTreeClassifier())])

    modelo_filtrado_knn=Pipeline([("filter",fea_selector),("model",KNeighborsClassifier())])
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    #print(f"Accuracy Score: {round(accuracy_score(y_train,predictions_dev)*100, 4)}%")
    print(f"Accuracy Score for DT Full features: {round(np.mean(sklearn.model_selection.cross_validate(estimator=DecisionTreeClassifier(), X=X, y=y, cv=5, n_jobs=4, scoring = 'accuracy')['test_score'])*100, 2)}%")
    print(f"Accuracy Score for DT filtered features: {round(np.mean(sklearn.model_selection.cross_validate(estimator=modelo_filtrado_dt, X=X, y=y, cv=5, n_jobs=4, scoring = 'accuracy')['test_score'])*100, 2)}%")
    
    print(f"Accuracy Score for KNN Full features: {round(np.mean(sklearn.model_selection.cross_validate(estimator=KNeighborsClassifier(), X=X, y=y, cv=5, n_jobs=4, scoring = 'accuracy')['test_score'])*100, 2)}%")
    print(f"Accuracy Score for KNN filtered features: {round(np.mean(sklearn.model_selection.cross_validate(estimator=modelo_filtrado_knn, X=X, y=y, cv=5, n_jobs=4, scoring = 'accuracy')['test_score'])*100, 2)}%")
    
    return score

In [ ]:
for i in list:
  for j in lista:
    modelo_filtrado_knn=Pipeline([("filter",fea_selector),("model",KNeighborsClassifier())])
    print(f"Accuracy Score for DT Full features: {round(np.mean(sklearn.model_selection.cross_validate(estimator=DecisionTreeClassifier(), X=X, y=y, cv=5, n_jobs=4, scoring = 'accuracy')['test_score'])*100, 2)}%")

2.2

In [ ]:
score=analyze_filtering_benefits (X_madelon,y_madelon)

Accuracy Score for DT Full features: 74.92%
Accuracy Score for DT filtered features: 73.81%
Accuracy Score for KNN Full features: 72.5%
Accuracy Score for KNN filtered features: 83.62%


In [ ]:
score=analyze_filtering_benefits (X_amazon,y_amazon)

Accuracy Score for DT Full features: 40.2%
Accuracy Score for DT filtered features: 41.73%
Accuracy Score for KNN Full features: 25.0%
Accuracy Score for KNN filtered features: 22.53%


**Madelon**

Una vez ejecutado el cross validation para la full data y la filtered data se evidencia que en el DT la pérdida en el accuracy es de ~1%. Sin embargo, al usar el 10% con mayor puntaje de anova para el KNN, hace que incremente el accuracy. 

**Amazon**

Para el caso de Amazon, el resultado es contrario al de Madelon. 
Al comparar las ejecuciones para DT, usar el 10% de los atributos con mayor anova resulta en un ganancia marginal de Accuracy. Sin embargo, para el KNN resulta en una pérdida del 2.5% de la Full Data a la Filtered.

Para ambos data sets, la pérdida marginal (y en los casos excepcionales la ganancia) son un argumento a favor de filtrar atributos para mejorar la rápidez del procesamiento sin perder accuracy del modelo.

#**Exercise 3**

1. Write a function wrapping(learner, X, y, train_size, repeats, scoring, tries)that will randomly create tries subsets of attributes in X and evaluate the learner based on that sub-database using MCCV with the given train size and repeats, using the given scoring function. For each feature set, memorize the mean value of the scoring function. 
Return a pandas dataframe with m + 1 columns, where m is the number of attributes of X. There should be one line for every tried configuation, with the observed performance in the last cell. The other columns should be filled with 0/1 depending on whether anattribute was selected or not. 

2. Write a function analyze_wrapping_benefits(X, y)in which you create an analy-sis/test fold of X/y, and use (i) kNN and (i) DT with wrapping on the analysis set to find the best feature set using wrapping. Compare the performance of each of them. 

a) on the full analysis data and the projected analysis data. 

b) on the full test data and the projected test data.

This should give 4 numbers in total per dataset (of course the results on the test datamust not be used for decision making), so this isnota function that you could use insideyour analysis (except that you separated test data before).

3. Load the datasets madelon and amazon-commerce-reviewsfromopenml.org. You can use the API from openml.org.Run the wrapping analysis on both datasets. Interpret your observations!

In [ ]:
def wrapping(learner, X, y, train_size, repeats, scoring, tries):
    scores = []
    scores_r = []
    df=pd.DataFrame(data=list(X))
    for i in range(1,repeats+1):
        #iris_X=pd.DataFrame(data=X)
        value=random.randint(1, X.shape[1])
        sample= X.sample(n=value,axis='columns')
        scores_r = []
        for j in range(tries):
            X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=train_size)
            clf = learner().fit(X_train, y_train)
            scores_r.append(scoring(y_train,clf.predict(X_train)))
        score_f=sum(scores_r)/tries
        scores.append(score_f)
        df1=pd.DataFrame(data=list(sample))
        df1["Rep"+str(i)]=np.ones(sample.shape[1])
        df=df.merge(df1,on=0,how='left').fillna(0)
    df=df.T
    df.columns=list(X)
    df=df.drop(df.index[0])
    df["scores"]=scores
    return df

In [ ]:
df_knn=wrapping(KNeighborsClassifier,X_mushroom, y_mushroom, 0.75, 10, sklearn.metrics.matthews_corrcoef, 10)
df_dt=wrapping(DecisionTreeClassifier,X_mushroom, y_mushroom, 0.75, 10, sklearn.metrics.matthews_corrcoef, 10)
df=pd.concat([df_knn,df_dt])

In [ ]:
def analyze_wrapping_benefits(X, y):
  df_knn=wrapping(KNeighborsClassifier,X, y, 0.75, 5, sklearn.metrics.accuracy_score, 3)
  df_dt=wrapping(DecisionTreeClassifier,X, y, 0.75, 5, sklearn.metrics.accuracy_score, 3)
  df=pd.concat([df_knn,df_dt])
  df1=df
  df1["scores1"]=df1["scores"].astype(str).str[:5]
  df1["scores1"]=df1["scores1"].astype(float)
  df1=df1.drop('scores',axis=1)
  score_max=df1['scores1'].max()
  df1=df1.loc[df1['scores1']==score_max]
  df_max=df
  df_max=df_max.merge(df1[["scores1"]],left_index=True, right_index=True)
  df_max=df.loc[df['scores1']==score_max]
  df_max=df_max.drop("scores1",axis=1)
  df_max["Q_att"]=df_max.sum(axis=1)
  min_a=df_max["Q_att"].min()
  df_f=df_max[df_max["Q_att"]==min_a]
  mejor_rep=(df_f.T).reset_index()
  mejor_rep.columns=["Atributo","Marca_A"]
  mejor_rep=mejor_rep[mejor_rep["Marca_A"]==1]
  print(mejor_rep)
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
  var=np.unique(mejor_rep["Atributo"]).tolist()
  print(len(var))
  Xtr_pr=X_train[X_train.columns & var]
  Xte_pr=X_test[X_test.columns & var]
  clf_knn = KNeighborsClassifier().fit(X_train, y_train)
  clf_knn_pr = KNeighborsClassifier().fit(Xtr_pr, y_train)
  print(f"Accuracy Score Full Train Data: {round(accuracy_score(y_train,clf_knn.predict(X_train))*100, 4)}% with {len(X.columns)} attributes")
  print(f"Accuracy Score Full Test Data: {round(accuracy_score(y_test,clf_knn.predict(X_test))*100, 4)}% with {len(X.columns)} attributes")
  print(f"Accuracy Score Projected Train Data: {round(accuracy_score(y_train,clf_knn_pr.predict(Xtr_pr))*100, 4)}% with {len(var)} attributes")
  print(f"Accuracy Score Projected Test Data: {round(accuracy_score(y_test,clf_knn_pr.predict(Xte_pr))*100, 4)}% with {len(var)} attributes")
analyze_wrapping_benefits(X_mushroom, y_mushroom)

                                 Atributo Marca_A
2     cap-shape_c_0.0_1.0_1.0_1.0_1.0_1.0     1.0
5     cap-shape_f_1.0_1.0_1.0_1.0_1.0_1.0     1.0
6     cap-shape_k_0.0_1.0_1.0_1.0_1.0_1.0     1.0
9     cap-shape_s_1.0_1.0_1.0_1.0_1.0_1.0     1.0
11    cap-shape_x_1.0_1.0_1.0_1.0_1.0_1.0     1.0
..                                    ...     ...
212  population_s_0.0_1.0_1.0_1.0_1.0_1.0     1.0
213  population_s_1.0_1.0_1.0_1.0_1.0_1.0     1.0
224     habitat_m_0.0_1.0_1.0_1.0_1.0_1.0     1.0
231     habitat_w_1.0_1.0_1.0_1.0_1.0_1.0     1.0
232                                scores     1.0

[62 rows x 2 columns]
62
Accuracy Score Full Train Data: 100.0% with 232 attributes
Accuracy Score Full Test Data: 100.0% with 232 attributes
Accuracy Score Projected Train Data: 99.9824% with 62 attributes
Accuracy Score Projected Test Data: 99.959% with 62 attributes


In [ ]:
analyze_wrapping_benefits(X_madelon, y_madelon)

    Atributo Marca_A
1         V2     1.0
2         V3     1.0
3         V4     1.0
4         V5     1.0
5         V6     1.0
..       ...     ...
496     V497     1.0
497     V498     1.0
498     V499     1.0
499     V500     1.0
500   scores     1.0

[338 rows x 2 columns]
338
Accuracy Score Full Train Data: 82.3529% with 500 attributes
Accuracy Score Full Test Data: 72.3431% with 500 attributes
Accuracy Score Projected Train Data: 81.5833% with 338 attributes
Accuracy Score Projected Test Data: 67.6056% with 338 attributes


In [ ]:
analyze_wrapping_benefits(X_amazon, y_amazon)

      Atributo Marca_A
2           V3     1.0
4           V5     1.0
6           V7     1.0
9          V10     1.0
10         V11     1.0
...        ...     ...
9984     V9985     1.0
9989     V9990     1.0
9990     V9991     1.0
9991     V9992     1.0
10000   scores     1.0

[4127 rows x 2 columns]
4127
Accuracy Score Full Train Data: 54.9524% with 10000 attributes
Accuracy Score Full Test Data: 23.3333% with 10000 attributes
Accuracy Score Projected Train Data: 51.7143% with 4127 attributes
Accuracy Score Projected Test Data: 22.4444% with 4127 attributes


**Madelon**

Cuando se ejecuta la selección del set de atributos con mayor accuracy y buscando reducir la cantidad de atributos usados (para ayudar al procesamiento escalable), se nota una pérdida menor al 1% sobre el train data y menor al 4% sobre la test data. La decisión de incluir 338 (óptimo) o el total de 500 atributos, dependerá de la disposición que se tengan en recursos para ejecución y se quiera sumir su uso para obtener la ganancia marginal.

**Amazon**

Con la ejecución de Amazon el resultado es similar, usar un set de datos con menos del 42% del total de atribustos genera una pérdida (tanto en la train data como en la test data) de menos del 1%. En este caso la recomendación es conservar el modelo ejecutado con el subset óptimo siempre que el resultado es en la práctica el mismo. 

En ambas ejecuciones el resultado es similar. El trade off entre tener un modelo más sencillo y aumentar las metricas de performance, sugieren que es válido pensar en incluir menos atributos siempre que la pérdida en performance no es representativa en general.  